In [3]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import warnings
warnings.filterwarnings('ignore')

conn = sql.connect("ncaa_pbp.db")
df = pd.read_sql("""
                SELECT
                    EventID, ElapsedSeconds, EventType
                From
                    "2016-2017"
                Where
                    EventType = "made1_free" or
                    EventType = "miss1_free"
                Limit
                    1000
                 """, conn, index_col='EventID')

In [4]:
temp = df.shift(1)
temp = temp.join(df, lsuffix='_prev')
temp = temp.join(df.shift(-1), rsuffix='_2')
temp = temp.join(df.shift(-2), rsuffix='_3')
df = temp

In [5]:
df = df[(df['ElapsedSeconds_prev'] != df['ElapsedSeconds'])]

df.drop(['ElapsedSeconds_prev', 'EventType_prev'], axis=1, inplace=True)

In [6]:
df['EventType'] = df['EventType'] == 'made1_free'
df['EventType_2'] = df['EventType_2'] == 'made1_free'
df['EventType_3'] = df['EventType_3'] == 'made1_free'

In [7]:
df[(df['EventType']) & (~df['EventType_2'])]

,ElapsedSeconds,EventType,ElapsedSeconds_2,EventType_2,ElapsedSeconds_3,EventType_3
EventID,,,,,,
20434759,633,True,736.0,False,736.0,True
20435576,2260,True,2360.0,False,2379.0,True
20436315,767,True,894.0,False,894.0,True
20436910,1020,True,1119.0,False,1119.0,True
20437132,1958,True,2009.0,False,2009.0,True
20437566,1454,True,1533.0,False,1556.0,True
20437719,2049,True,2147.0,False,2147.0,True
20438202,1861,True,1963.0,False,2008.0,False
20438476,985,True,1057.0,False,1228.0,False


In [8]:
oneFT = df[(df['ElapsedSeconds'] != df['ElapsedSeconds_2']) &
           (df['ElapsedSeconds'] != df['ElapsedSeconds_3'])]
twoFT = df[(df['ElapsedSeconds'] == df['ElapsedSeconds_2']) &
           (df['ElapsedSeconds'] != df['ElapsedSeconds_3'])]
threeFT = df[(df['ElapsedSeconds'] == df['ElapsedSeconds_2']) &
           (df['ElapsedSeconds'] == df['ElapsedSeconds_3'])]
oneFT.drop(oneFT.columns[2:], axis=1, inplace=True)
twoFT.drop(twoFT.columns[4:], axis=1, inplace=True)
threeFT.drop(threeFT.columns[6:], axis=1, inplace=True)

In [9]:
_0_1 = twoFT[(~twoFT['EventType']) & (twoFT['EventType_2'])]
_1_1 = twoFT[(twoFT['EventType']) & (twoFT['EventType_2'])]
_0_0 = twoFT[(~twoFT['EventType']) & (~twoFT['EventType_2'])]
_1_0 = twoFT[(twoFT['EventType']) & (~twoFT['EventType_2'])]
_0 = oneFT[~oneFT['EventType']]
_1 = oneFT[oneFT['EventType']]
_0_0_0 = threeFT[(~threeFT['EventType']) & (~threeFT['EventType_2']) & (~threeFT['EventType_3'])]
_1_0_0 = threeFT[(threeFT['EventType']) & (~threeFT['EventType_2']) & (~threeFT['EventType_3'])]
_0_1_0 = threeFT[(~threeFT['EventType']) & (threeFT['EventType_2']) & (~threeFT['EventType_3'])]
_0_0_1 = threeFT[(~threeFT['EventType']) & (~threeFT['EventType_2']) & (threeFT['EventType_3'])]
_1_1_0 = threeFT[(threeFT['EventType']) & (threeFT['EventType_2']) & (~threeFT['EventType_3'])]
_1_0_1 = threeFT[(threeFT['EventType']) & (~threeFT['EventType_2']) & (threeFT['EventType_3'])]
_0_1_1 = threeFT[(~threeFT['EventType']) & (threeFT['EventType_2']) & (threeFT['EventType_3'])]
_1_1_1 = threeFT[(threeFT['EventType']) & (threeFT['EventType_2']) & (threeFT['EventType_3'])]

In [10]:
seqs = [_0, _1, _0_0, _1_0, _0_1, _1_1,   
    _0_0_0, _1_0_0, _0_1_0, _0_0_1, 
    _1_1_0, _1_0_1, _0_1_1, _1_1_1]
for i, data in enumerate(seqs):
    data['ftCode'] = i

In [11]:
finalDF = pd.concat(seqs)

In [12]:
set(finalDF['ftCode'])

{0, 1, 4, 5, 9, 12, 13}

In [13]:
from collections import Counter
Counter(list(zip(twoFT['EventType'], twoFT['EventType_2'])))

Counter({(True, True): 221, (False, True): 168})

In [32]:
original = pd.read_sql("""
                SELECT
                    *
                From
                    "2016-2017"
                Limit
                    1000
                 """, conn, index_col='EventID')
finalDF = finalDF['ftCode']

In [34]:
original['ftCode'] = finalDF

In [37]:
toRem = original[(original['EventType'] == 'made1_free') |
                (original['EventType'] == 'made1_free')]
toRem = toRem[~toRem.index.isin(finalDF.index)]

In [39]:
original = original[~original.index.isin(toRem.index)]

In [40]:
original

,index,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,ftCode
EventID,,,,,,,,,,,,
20434640,0,2017,11,1104,1157,0,0,0,1104,637419,sub_in,NaN
20434641,1,2017,11,1104,1157,0,0,15,1157,638145,foul_pers,NaN
20434642,2,2017,11,1104,1157,0,0,15,1157,638145,turnover,NaN
20434643,3,2017,11,1104,1157,0,0,26,1104,637423,assist,NaN
20434644,4,2017,11,1104,1157,3,0,26,1104,637420,made3_jump,NaN
20434645,5,2017,11,1104,1157,0,0,63,1157,638145,miss2_jump,NaN
20434646,6,2017,11,1104,1157,0,0,63,1104,637415,reb_def,NaN
20434647,7,2017,11,1104,1157,0,0,68,1104,637410,miss3_jump,NaN
20434648,8,2017,11,1104,1157,0,0,68,1157,638150,reb_def,NaN
